<a href="https://colab.research.google.com/github/JaebinChoi/Recommendation-System/blob/master/RMSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [ ]:
import pandas as pd
pd.__version__

'1.0.5'

In [ ]:
path = '/gdrive/My Drive/Recommend'
ratings = pd.read_pickle('%s/data/ml-latest-small/ratings_updated.p' % path)

In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04
3,1,47,5.0,2000-07-30 19:03:35
4,1,50,5.0,2000-07-30 18:48:51
...,...,...,...,...
23,1000,2,4.0,2020-08-26 13:16:00
24,1000,122920,4.0,2020-08-26 13:16:00
25,1000,110102,3.7,2020-08-26 13:16:00
26,1000,159858,4.0,2020-08-26 13:16:00


### 평가지표 RMSE

In [ ]:
rating_example = [[4, 3.5], [5, 5], [0.5, 1], [3, 5]]
rating_example = pd.DataFrame(rating_example, columns=['Actual', 'Predict'])

In [ ]:
# error = Actual - Predict
rating_example['error'] = rating_example['Actual'] - rating_example['Predict']

In [ ]:
# squared error : + 와 - 가 캔슬되는 것(0.5 - 0.5 = 0)을 막기 위해 제곱
rating_example['squared error'] = rating_example['error'] ** 2

In [ ]:
# mean squared error : mse
mse = rating_example['squared error'].mean()
mse

1.125

In [ ]:
import numpy as np

In [ ]:
# root mean squared error : rmse
rmse = np.sqrt(mse)
rmse

1.0606601717798212

### RMSE with sklearn

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mse = mean_squared_error(rating_example['Actual'], rating_example['Predict'])
rmse = np.sqrt(mse)
rmse
# 평균적으로 1점 정도 틀리다

1.0606601717798212

### Train Task Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# random_state 임의값, test_size(0.1 = 10%)
train, test = train_test_split(ratings, random_state=42, test_size=0.1)

In [ ]:
print(train.shape)
print(test.shape)

(90777, 4)
(10087, 4)


### 가장 간단한 예측하기

In [ ]:
test.shape

(10087, 4)

* 모두 0.5점으로 예측하기

In [ ]:
predictions = [0.5] * len(test)

In [ ]:
# mean_squared_error(actual, predict)
mse = mean_squared_error(test['rating'], predictions)
rmse = np.sqrt(mse)
rmse
# 평균적으로 3점 정도 틀리다

3.1642403468037372

* 모두 5점으로 예측하기

In [ ]:
predictions = [5] * len(test)

mse = mean_squared_error(test['rating'], predictions)
rmse = np.sqrt(mse)
rmse

1.8492963248209124

* 데이터의 평균으로 예측하기

  주의점 : train 데이터의 평균으로 test 데이터의 평균을 예측해야 함

In [ ]:
predictions = [train['rating'].mean()] * len(test)

mse = mean_squared_error(test['rating'], predictions)
rmse = np.sqrt(mse)
rmse

1.0571439079877267

* 사용자 평점 기반 예측하기:
  - 각 사용자의 평균 평점(train)으로 영화에 대한 평점을 예측(test)하세요.
  - train에 해당 사용자에 대한 평점기록이 전혀 없다면, 어떻게 해야 할까요?
  - (예: userId 1번의 train 데이터상 평균 평점을 4.33이라고 한다면, userId 1번의 test 데이터에 대한 모든 영화 평점을 4.33으로 한다.)
  - 이 예측 방법은 어떤 가정을 가지고 있나요? 예측의 근거를 찾아보세요.

In [ ]:
users = train.groupby('userId')['rating'].mean().reset_index() # reset_index() : 표 형식으로 변경

# rename : 컬럼명을 바꿔준다 ! rating -> predict (dictionary 형식)
users = users.rename(columns={'rating' : 'predict'})
users[:5]

,userId,predict
0,1,4.333333
1,2,3.925926
2,3,2.529412
3,4,3.512690
4,5,3.627907


In [ ]:
# 두 테이블을 붙임, how : 조인 방식(inner, left, right, outer), on : 기준
predict_by_users = test.merge(users, how='left', on='userId')
predict_by_users

,userId,movieId,rating,timestamp,predict
0,606,1227,4.0,2007-02-11 23:32:45,3.664336
1,103,1348,4.0,2015-05-18 17:26:13,3.909357
2,155,432,4.0,2000-06-24 23:23:27,3.619048
3,199,44195,4.0,2009-12-21 02:46:19,3.365741
4,553,5989,5.0,2009-01-01 01:37:29,4.331169
...,...,...,...,...,...
10082,298,1587,3.0,2016-01-17 12:15:46,2.360915
10083,232,27816,2.5,2008-06-11 07:48:35,3.269330
10084,599,45208,1.5,2017-06-26 19:15:59,2.646030
10085,249,1432,3.5,2012-11-29 21:50:55,3.685638


In [ ]:
predict_by_users.isnull().sum() # null 개수

userId       0
movieId      0
rating       0
timestamp    0
predict      0
dtype: int64

In [ ]:
mse = mean_squared_error(predict_by_users['rating'], predict_by_users['predict'])
rmse = np.sqrt(mse)
rmse

0.9553815413196951

예측의 근거

In [ ]:
train['rating'].std() # 전체의 표준편차
# 모든 데이터의 평균에서 1점정도 떨어져있다.

1.0408547822589538

In [ ]:
# 유저 별 표준편차
train.groupby('userId')['rating'].std().mean()

# 유저 별 표춘편차의 평균이 전체의 표준편차보다 작음 => 유저 별 표준편차를 사용하는 것이 더 효율적

0.925211328941015

* 영화 평점 기반 예측하기:
  - 각 영화의 평균 평점(train)으로 영화에 대한 평점을 예측(test)하세요.
  - train에 해당 영화에 대한 평점기록이 전혀 없다면, 어떻게 해야 할까요?
  - (예: movieId 1번의 train 데이터상 평균 평점을 4.5라고 한다면, movieId 1번의 test 데이터에 대한 모든 영화 평점을 4.5로 한다.)
  - 이 예측 방법은 어떤 가정을 가지고 있나요? 예측의 근거를 찾아보세요.

In [ ]:
movies = train.groupby('movieId')['rating'].mean().reset_index()
movies = movies.rename(columns={'rating' : 'predict'})

movies[:5]

,movieId,predict
0,1,3.889474
1,2,3.410891
2,3,3.218750
3,4,2.300000
4,5,3.104651


In [ ]:
predict_by_movie = test.merge(movies, how='left', on='movieId')
predict_by_users[:5]

,userId,movieId,rating,timestamp,predict
0,606,1227,4.0,2007-02-11 23:32:45,3.664336
1,103,1348,4.0,2015-05-18 17:26:13,3.909357
2,155,432,4.0,2000-06-24 23:23:27,3.619048
3,199,44195,4.0,2009-12-21 02:46:19,3.365741
4,553,5989,5.0,2009-01-01 01:37:29,4.331169


In [ ]:
predict_by_movie.isnull().sum()

userId         0
movieId        0
rating         0
timestamp      0
predict      369
dtype: int64

In [ ]:
# NaN (Not a Number)
# Pandas location
# df.loc(index(filtering), column)
# test에 데이터가 없는 경우 train의 평균을 넣어준다
predict_by_movie.loc[predict_by_movie['predict'].isnull(), 'predict'] = train['rating'].mean()

In [ ]:
predict_by_movie.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
predict      0
dtype: int64

In [ ]:
mse = mean_squared_error(predict_by_movie['rating'], predict_by_movie['predict'])
rmse = np.sqrt(mse)
rmse

0.9887094919898134